In [ ]:
import requests
#Requests Library for downloading webpages from the web
url='https://github.com/topics'
response=requests.get(url)
#.get() is a method of requests library which is used to download the webpage.
response.status_code
#This will give the status code of the webpage and if it is 200 then it means that the webpage is downloaded successfully

In [ ]:
html_content=response.text
html_content
#This will give the html code of the webpage

In [ ]:
with open('webpage.html','w',encoding="utf-8") as f:
    f.write(html_content)
#This will create a html file of the webpage

In [ ]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(html_content,'html.parser')
#This will parse the html code of the webpage

In [ ]:

p_tags_all=soup.find_all('p')
#This will give the list of all the p tags in the webpage

p_name=soup.find_all('p',class_='f3 lh-condensed mb-0 mt-1 Link--primary')
print(p_name)
#This will give the list of all the p tags in the webpage whose class is f3 lh-condensed mb-0 mt-1 Link--primary

p_disc=soup.find_all('p',class_='f5 color-fg-muted mb-0 mt-1')
#This will give the list of all the p tags in the webpage whose class is f5 color-text-secondary mb-0 mt-1
print(p_disc)

p_link=soup.find_all('a',class_='no-underline flex-1 d-flex flex-column')
print(p_link)
#This will give the list of all the a tags in the webpage whose class is no-underline flex-1 d-flex flex-column
print("https://github.com"+p_link[0]['href'])
#This will give the link of the first topic in the webpage
#['href'] is used to get the link of the tag

In [ ]:

#Now we will create a list of all the topics,description & their link in the webpage
topic_name=[]
topic_disc=[]
topic_link=[]
for i in range(30):
    topic_name.append(p_name[i].text.strip())
    topic_disc.append(p_disc[i].text.strip())
    topic_link.append("https://github.com"+p_link[i]['href'])
print(topic_name)
print(topic_disc)
print(topic_link)

In [ ]:
#Now we will create a dataframe of all the topics,description & their link in the webpage
import pandas as pd
topics_df=pd.DataFrame({})
topics_df['Topic Name']=topic_name
topics_df['Topic Description']=topic_disc
topics_df['Topic Link']=topic_link
topics_df

In [8]:
topics_df.to_csv('gitHub.csv',index=None)

In [9]:
topic_page_url=topic_link[0]
response=requests.get(topic_page_url)
response.status_code
html_indi_content=response.text
soup=BeautifulSoup(html_indi_content,'html.parser')
#Repeating the same steps as above.

In [10]:
repo_tags=soup.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
user_name=repo_tags[0].find_all('a')[0].text.strip() #repo_tags[0] is used to get the first repo and find_all('a')[0] is used to get the first username
repo_name=repo_tags[0].find_all('a')[1].text.strip() #repo_tags[0] is used to get the first repo and find_all('a')[1] is used to get the first repo name
repo_link="https://github.com"+repo_tags[0].find_all('a')[1]['href'] 



In [11]:
stars_tags=soup.find_all('span',class_='Counter js-social-count')
text=stars_tags[0].text.strip()
def star_count(text):
    if text[-1]=='k':
        stars=int(float(text[:-1])*1000)
    else:
        stars=int(text)
    return stars

#This function is used to get the stars of the repo in integer format.


In [12]:
def repo_information(main_tag,main_star_tag):
    a_tags=main_tag.find_all('a')
    user_name=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_link="https://github.com"+a_tags[1]['href']
    stars=star_count(main_star_tag.text.strip())
    return user_name,repo_name,repo_link,stars
#Putting it all together

In [ ]:
repo_information(repo_tags[0],stars_tags[0])

In [14]:
repo_username=[]
repo_name=[]
repo_link=[]
repo_stars=[]
for i in range(20):
    repo_info=repo_information(repo_tags[i],stars_tags[i])
    repo_username.append(repo_info[0])
    repo_name.append(repo_info[1])
    repo_link.append(repo_info[2])
    repo_stars.append(repo_info[3])
    

In [ ]:
df=pd.DataFrame({'Username':repo_username,'Repository Name':repo_name,'Repository Link':repo_link,'Stars':repo_stars})
df


In [16]:
#Now we will create a function to get the information of all repositories in each topic in the webpage
def get_topic(url):
    response=requests.get(url)
    if response.status_code!=200:
        raise Exception('Failed to load page {}'.format(url))
    soup=BeautifulSoup(response.text,'html.parser')
    return soup

def repo_information(main_tag,main_star_tag):
    a_tags=main_tag.find_all('a')
    user_name=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_link="https://github.com"+a_tags[1]['href']
    stars=star_count(main_star_tag.text.strip())
    return user_name,repo_name,repo_link,stars

def topic_repo(soup):
    repo_tags=soup.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
    stars_tags=soup.find_all('span',class_='Counter js-social-count')
    topic_repos={
        'Username':[],
        'Repository Name':[],
        'Repository Link':[],
        'Stars':[]
}
    for i in range(len(repo_tags)):
        repo_info=repo_information(repo_tags[i],stars_tags[i])
        topic_repos['Username'].append(repo_info[0])
        topic_repos['Repository Name'].append(repo_info[1])
        topic_repos['Repository Link'].append(repo_info[2])
        topic_repos['Stars'].append(repo_info[3])
    return pd.DataFrame(topic_repos)

In [ ]:
topic_repo(get_topic(topic_link[6]))
#This will give the information of the third topic in the webpage

In [18]:
import os
def scrape_topic_csv(topic_url,path):
    if os.path.exists(path):
        print('File {} already exists'.format(path))
        return
    topic_df=topic_repo(get_topic(topic_url))
    topic_df.to_csv(path,index=None)
    

In [19]:
def scrape_topics():
    topics_url='https://github.com/topics'
    response=requests.get(topics_url)
    if response.status_code!=200:
        raise Exception('Failed to load page {}'.format(topics_url))
    soup=BeautifulSoup(response.text,'html.parser')
    
    p_name=soup.find_all('p',class_='f3 lh-condensed mb-0 mt-1 Link--primary')
    p_disc=soup.find_all('p',class_='f5 color-fg-muted mb-0 mt-1')
    p_link=soup.find_all('a',class_='no-underline flex-1 d-flex flex-column')
    topic_name=[]
    topic_disc=[]
    topic_link=[]
    for i in range(len(p_name)):
        topic_name.append(p_name[i].text.strip())
        topic_disc.append(p_disc[i].text.strip())
        topic_link.append("https://github.com"+p_link[i]['href'])
    topics_dictt={
        'Topic Name':topic_name,
        'Topic Description':topic_disc,
        'Topic Link':topic_link

    }
    return pd.DataFrame(topics_dictt)    


In [ ]:

        
def scrape_topics_repos():
    topics_df=scrape_topics()
    os.makedirs('Topics',exist_ok=True)
    for index,row in topics_df.iterrows():
        print('Scraping {} topic'.format(row['Topic Name']))
        scrape_topic_csv(row['Topic Link'],'Topics/{}.csv'.format(row['Topic Name']))
scrape_topics_repos()
        
        
        